In [8]:
import cudaq
import numpy as np
import scipy
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [9]:
mnist = fetch_openml('mnist_784', version=1, cache=True)
data = mnist['data']
labels = np.array(mnist['target'], dtype=np.int8)

labels_zero = labels[labels==0] 
labels_one = labels[labels==1] 
binary_labels = np.hstack((labels_zero, labels_one))
digits_zero = data[labels==0]
digits_one = data[labels==1]
binary_digits = np.vstack((digits_zero, digits_one))
        
pca = PCA(n_components=4)
sc = StandardScaler()
binary_digits = sc.fit_transform(binary_digits)
data = pca.fit_transform(binary_digits)
data = (data-np.min(data))/(np.max(data)-np.min(data))

x_train, x_test, y_train, y_test = train_test_split(data, binary_labels, test_size=0.2)

x_train=x_train[0:500]
x_test=x_test[0:200]
y_train=y_train[0:500]
y_test=y_test[0:200]

In [10]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((500, 4), (200, 4), (500,), (200,))

In [11]:

cudaq.set_target("nvidia")
qubit_count = 4

@cudaq.kernel
def encoding(qubit:cudaq.qvector, angles:list[float]):
    # Define gates and the qubits they act upon.

    qubit_num=qubit.size()
    for i in range(qubit_num):
        ry(angles[i], qubit[i])
    
    for i in range(qubit_num-1):
        x.ctrl(qubit[i], qubit[i+1])

@cudaq.kernel
def training_layer(qubit:cudaq.qvector, alpha:list[float]):

    qubit_num=qubit.size()

    count=0
    for i in range(qubit_num):
        u3(alpha[count],alpha[count+1],alpha[count+2], qubit[i])
        count+=3
    
    for i in range(qubit_num-1):
        x.ctrl(qubit[i], qubit[i+1])
    
    x.ctrl(qubit[qubit_num-1], qubit[0])

    for i in range(qubit_num):
        u3(alpha[count],alpha[count+1],alpha[count+2], qubit[i])
        count+=3

    for i in range(qubit_num//2):
        x.ctrl(qubit[i], qubit[i+2])
    
    for i in range(qubit_num-1, qubit_num//2-1, -1):
        x.ctrl(qubit[i], qubit[i-2])

@cudaq.kernel
def main_kernel(qubit_count:int, angles: list[float],alpha:list[float]):
    # Allocate a qubit that is initialised to the |0> state.
    qubit = cudaq.qvector(qubit_count)

    # encoding data
    encoding(qubit,angles)
    
    # training layer
    training_layer(qubit,alpha)


# Our hamiltonian will be the Z expectation value of our qubit.
ham = cudaq.spin.z(0)


In [12]:
np.random.seed(42)
alpha=np.random.normal(0, np.pi, 3*(qubit_count*2))
alpha=alpha.tolist()

bias=[0.0]
alpha=alpha+bias

print(alpha)
n=len(alpha)
print(alpha[n-1])


[1.5604735340341649, -0.4343701128131203, 2.034773553111448, 4.78473940808937, -0.7356145218440903, -0.7355629438854153, 4.961243379652874, 2.410967307216451, -1.474897281901825, 1.7045026470610245, -1.4558696192831628, -1.4631331723745038, 0.7601468947977512, -6.01074716087542, -5.418989190668973, -1.7664783710685958, -3.1819028069697457, 0.9872371114915007, -2.8526417649400995, -4.436882932752625, 4.604471405186879, -0.709297166963182, 0.21214611175768935, -4.4759784350235785, 0.0]
0.0


In [13]:
def cost(main_kernel, ham,qubit_count,angles,alpha):
    expectation_value = cudaq.observe(main_kernel, ham, qubit_count,angles,alpha).expectation()
    return expectation_value

def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [14]:

def loss(alpha, main_kernel, ham, qubit_count, x_train):
    pred=[]

    param=alpha[0:n-2]
    for image in range (10):
    
        eig=cost(main_kernel, ham, qubit_count, x_train[image], param)
        pred.append(eig+alpha[n-1])

    #print(pred)
    #print(y_train[:10])

    return square_loss(y_train[:10],pred)

def variational_classifier(alpha,main_kernel,ham,qubit_count,data_img):
    
    param=alpha[0:n-2]
    
    pred=cost(main_kernel, ham, qubit_count,data_img, param)+alpha[n-1]
    
    return pred

#res=loss(alpha, main_kernel, ham, qubit_count, x_train)
#print(res)

In [15]:

result_opt=scipy.optimize.minimize(loss, alpha, method='COBYLA', args=(main_kernel, ham, qubit_count, x_train), tol=1e-5)

#print('Final loss', result_opt.fun)
#print('Opt. parameters', result_opt.x)
alpha=result_opt.x

predictions = [np.sign(variational_classifier(alpha,main_kernel,ham,qubit_count,x_train[i])) for i in range(10)]
print(predictions)
   
acc = accuracy(y_train[:10], predictions)

c=loss(alpha, main_kernel, ham, qubit_count, x_train)

print('Cost: ', c)
print('accuracy: ', acc)



[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Cost:  0.10558979009100877
accuracy:  0.6
